## Predict Deep Learning Model

Predict the sentiment analysis label, using a deep learning model

In [1]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

In [2]:
import pickle
import pandas as pd
from pathlib import Path

In [3]:
model_weights_path = Path('../models/model_weights.h5').resolve()
dataset_path = Path(
    '../datasets/tweepy/1148934575885869056-1145529100691476482_#excited.csv'
).resolve()
tokenizer_file = Path('../datasets/sentiment140/tokenizer.pickle').resolve()

In [4]:
with tokenizer_file.open('rb') as file:
    tokenizer = pickle.load(file)

In [5]:
from sentiment_analysis.models.lstm_model import lstm_model

In [6]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
model = lstm_model(100, input_dim, embedding_dim=200)
model.load_weights(model_weights_path.as_posix())

W0711 15:56:42.200845 4592022976 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 15:56:42.235319 4592022976 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 15:56:42.242611 4592022976 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_op

In [7]:
dataset = pd.read_csv(dataset_path)

In [8]:
from nlp.utils import preprocess
cleaned_texts = preprocess(dataset.text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rmohashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Time to clean up: 2.14 sec


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
predict_sequences = [text.split() for text in cleaned_texts]
list_tokenized_predict = tokenizer.texts_to_sequences(predict_sequences)
x_predict = pad_sequences(list_tokenized_predict, maxlen=100)

In [23]:
result = model.predict(x_predict)

In [24]:
import numpy as np

In [25]:
mean = np.mean(result)
std = np.std(result)

In [31]:
len(result)

3995

In [36]:
reduced_result = result[np.all([(result >= mean - std), (result <= mean + std)], axis=0)]

In [37]:
len(reduced_result)

3553